In [ ]:
from PIL import Image
import numpy as np
from pathlib import Path

In [43]:
def read_ppm(ppm_path: str) -> np.ndarray:
    with open(ppm_path, 'rb') as ppm:
        header = ppm.readline().decode()
        if header.strip() != 'P6':
            raise Exception('Only Raw PPM file supported.')
        
        metadata_count = 0
        metadata = []
        while metadata_count < 2:
            line = ppm.readline().decode()
            if line.startswith('#'):
                continue
            
            metadata.extend([int(v) for v in line.split(' ')])
            metadata_count += 1
            
        width, height, maxval = metadata

        if maxval > 255:
            raise Exception('Only 8-bit images supported.')


        return np.frombuffer(ppm.read(), dtype=np.uint8).reshape((height, width, 3))

def apply_filter(img_channel: np.ndarray, filter_size: int, filt: np.ndarray) -> np.ndarray:
    filt = np.flip(filt, axis=(0,1))
    pad_size = filter_size // 2
    padded_img = np.pad(img_channel.astype(np.float32), pad_size, mode='constant')
    height, width = padded_img.shape

    for i in range(pad_size, height - pad_size):
        for j in range(pad_size, width - pad_size):
            chunk = padded_img[i - pad_size:i + pad_size + 1, j - pad_size:j + pad_size + 1]
            padded_img[i, j] = np.sum(chunk * filt)

    output = padded_img[pad_size:height - pad_size, pad_size:width - pad_size]
    return np.clip(output, 0, 255).astype(np.uint8)


def filter_ppm(img: np.ndarray, filter_size: int, filt: np.ndarray) -> np.ndarray:
    filtered_channels = []
    for c in range(3):
        filtered_channels.append(apply_filter(img[:, :, c], filter_size, filt))
    
    return np.stack(filtered_channels, axis=-1)

def get_identity_kernel(size: int) -> np.ndarray:
    if size % 2 == 0:
        raise Exception("Kernel must have an odd size.")
    if size <= 1:
        raise Exception("Kernels must have size 3 or greated (and should be odd).")

    identity = np.zeros((size, size), dtype=np.float32)
    identity[size//2, size//2] = 1
    return identity

def save_ppm(ppm_image: np.ndarray, filename: str, foldername:str='./output') -> None:
    folder = Path(foldername)
    folder.mkdir(exist_ok=True, parents=True)

    filepath = folder / Path(filename)

    header = 'P6'
    height, width, _ = ppm_image.shape
    maxval = int(np.max(ppm_image))

    ppm_image = np.clip(ppm_image, 0, 255).astype(np.uint8)

    print(width, height)
    with open(filepath, 'wb') as ppm:
        ppm.write(f"{header}\n".encode('ascii'))
        ppm.write(f"{width} {height}\n".encode('ascii'))
        ppm.write(f"{maxval}\n".encode('ascii'))
        ppm.write(ppm_image.tobytes())

# Referência: 
# https://stackoverflow.com/questions/29731726/how-to-calculate-a-gaussian-kernel-matrix-efficiently-in-numpy
def get_gaussian_kernel(size=5, sigma=1):
    """
    Creates gaussian kernel with side length `l` and a sigma of `sig`
    """
    ax = np.linspace(-(size - 1) / 2.0, (size - 1) / 2.0, size)
    gauss = np.exp(-0.5 * np.square(ax) / np.square(sigma))
    kernel = np.outer(gauss, gauss)
    return kernel / np.sum(kernel)

In [ ]:
img_path = '../ppm_images/west_1.ppm'
img = read_ppm(img_path)
# img_path = '../images/mountain.jpg'
# img = np.array(Image.open(img_path))
img.shape

n = 5
mean_kernel = 1 / 9 * np.ones((n, n), dtype=np.float32)
identity = get_identity_kernel(n) 
gauss_kernel = get_gaussian_kernel(n)

sharp_basic = 2 * identity - mean_kernel
sharp_gauss = 2 * identity - gauss_kernel

filtered = filter_ppm(img, n, sharp_gauss)
save_ppm(filtered, 'sharp_gauss_west_1_2.ppm', '../output')

366 216


In [ ]:
# Show filtered
fi = Image.fromarray(filtered)
fi.show()

# Show Original
with Image.open(img_path, 'r') as img:
    img.show()

np.uint8(255)